In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [72]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,slight,thin,peculiarly,splash,chance,funny,relinquish,help,lonely,liberality,...,curate,uncommon,post,pennyworth,consistent,gratitude,gifted,only,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [7]:
word_counts['str_sentence'] = [i.text for i in word_counts.text_sentence]

In [12]:
word_counts['sent_len'] = [len(i) for i in word_counts['str_sentence']]

In [25]:
print(word_counts.loc[0,'text_sentence'])
print([x.pos_ for x in word_counts.loc[0,'text_sentence']])

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
['PROPN', 'VERB', 'VERB', 'PART', 'VERB', 'ADV', 'ADJ', 'ADP', 'VERB', 'ADP', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'VERB', 'NOUN', 'PART', 'VERB', 'PUNCT', 'ADV', 'CCONJ', 'ADV', 'PRON', 'VERB', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'VERB', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'PUNCT', 'CCONJ', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PUNCT', 'VERB', 'PROPN', 'PUNCT', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'PUNCT']


In [26]:
# How many of each type of word is there?
word_counts['cnt_verbs'] = [sum([1 for x in j if x.pos_ == 'VERB']) for j in word_counts.text_sentence]
word_counts['cnt_adj'] = [sum([1 for x in j if x.pos_ == 'ADJ']) for j in word_counts.text_sentence]
word_counts['cnt_prop'] = [sum([1 for x in j if x.pos_ == 'PROPN']) for j in word_counts.text_sentence]
word_counts['cnt_punct'] = [sum([1 for x in j if x.pos_ == 'PUNCT']) for j in word_counts.text_sentence]
word_counts['cnt_adv'] = [sum([1 for x in j if x.pos_ == 'ADV']) for j in word_counts.text_sentence]
word_counts['cnt_nouns'] = [sum([1 for x in j if x.pos_ == 'NOUN']) for j in word_counts.text_sentence]
word_counts['crude_sentiment'] = [j.sentiment for j in word_counts.text_sentence]

In [27]:
word_counts.head()

,slight,thin,peculiarly,splash,chance,funny,relinquish,help,lonely,liberality,...,text_source,str_sentence,sent_len,cnt_verbs,cnt_adj,cnt_prop,cnt_punct,cnt_adv,cnt_nouns,crude_sentiment
0,0,0,0,0,0,0,0,0,0,0,...,Carroll,Alice was beginning to get very tired of sitti...,301,13,3,2,10,3,12,0.0
1,0,0,0,0,0,0,0,0,0,0,...,Carroll,So she was considering in her own mind (as wel...,289,11,7,2,7,7,8,0.0
2,0,0,0,0,0,0,0,0,0,0,...,Carroll,There was nothing so VERY remarkable in that; ...,140,5,1,2,4,6,2,0.0
3,0,0,0,0,0,0,0,0,0,0,...,Carroll,Oh dear!,8,0,0,0,1,0,0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,Carroll,I shall be late!',17,2,1,0,2,0,0,0.0


In [28]:
#CHECK
word_counts.loc[4,'str_sentence']

"I shall be late!'"

In [29]:
word_counts['shall'].head()

0    0
1    0
2    0
3    0
4    1
Name: shall, dtype: object

In [30]:
word_counts['late'].head()

0    0
1    0
2    0
3    0
4    1
Name: late, dtype: object

### Modeling

In [31]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source', 'str_sentence'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9915360501567398

Test set score: 0.8745300751879699

Classification Report:
               precision    recall  f1-score   support

      Austen       0.86      0.97      0.91      1472
     Carroll       0.92      0.65      0.76       656

   micro avg       0.87      0.87      0.87      2128
   macro avg       0.89      0.81      0.84      2128
weighted avg       0.88      0.87      0.87      2128



Even with the added Features, the random forest model continues to overfit

In [32]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test, y_pred))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training set score: 0.9592476489028213

Test set score: 0.9210526315789473

Classification Report:
               precision    recall  f1-score   support

      Austen       0.92      0.98      0.94      1472
     Carroll       0.94      0.80      0.86       656

   micro avg       0.92      0.92      0.92      2128
   macro avg       0.93      0.89      0.90      2128
weighted avg       0.92      0.92      0.92      2128



Adding in the new features improved this model by a couple percentage points.

In [33]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test, y_pred))

Training set score: 0.893730407523511

Test set score: 0.8782894736842105

Classification Report:
               precision    recall  f1-score   support

      Austen       0.87      0.97      0.92      1472
     Carroll       0.91      0.67      0.77       656

   micro avg       0.88      0.88      0.88      2128
   macro avg       0.89      0.82      0.84      2128
weighted avg       0.88      0.88      0.87      2128



Similarly improved by a couple percentage points, it's inability to recall the Carroll work is somewhat disappointing but this model has the least overfitting, although the logistic regression still outperforms it.

### Challenge 01
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [37]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [73]:
#Can this differentiate Paradise from Alice in Wonderland?

wh = gutenberg.raw('whitman-leaves.txt')
wh = re.sub(r'VOLUME \w+', '', wh)
wh = re.sub(r'CHAPTER \w+', '', wh)
wh = text_cleaner(wh)
print(type(wh))
print(wh[:800])

<class 'str'>
Come, said my soul, Such verses for my Body let us write, (for we are one,) That should I after return, Or, long, long hence, in other spheres, There to some group of mates the chants resuming, (Tallying Earth's soil, trees, winds, tumultuous waves,) Ever with pleas'd smile I may keep on, Ever and ever yet the verses owning as, first, I here and now Signing for Soul and Body, set to them my name, Walt Whitman } One's-Self I Sing One's-self I sing, a simple separate person, Yet utter the word Democratic, the word En-Masse. Of physiology from top to toe I sing, Not physiognomy alone nor brain alone is worthy for the Muse, I say the Form complete is worthier far, The Female equally with the Male I sing. Of Life immense in passion, pulse, and power, Cheerful, for freest action form'd under the


In [74]:
len(wh)

692312

In [75]:
len(alice)

141708

In [57]:
wh_doc = nlp(wh)

In [76]:
wh_sents = [[sent, "Whitman"] for sent in wh_doc.sents]

In [78]:
wdf = pd.DataFrame(wh_sents)
wdf = wdf.rename({0:'text_sentence', 1:'text_source'}, axis=1)
wdf.head()

,text_sentence,text_source
0,"(Come, ,, said, my, soul, ,, Such, verses, for...",Whitman
1,"(That, should, I, after, return, ,, Or, ,, lon...",Whitman
2,"(Ever, with, pleas'd, smile, I, may, keep, on,...",Whitman
3,"(I, here, and, now, Signing, for, Soul, and, B...",Whitman
4,(One's),Whitman


In [79]:
wdf['str_sentence'] = [x.text for x in wdf['text_sentence']]

In [80]:
wh_words = bag_of_words(wh_doc)

In [81]:
common_words2 = set(alicewords + wh_words)

In [82]:
sentences2 = pd.DataFrame(alice_sents + wh_sents)

In [83]:
word_counts2 = bow_features(sentences2, common_words2)
word_counts2.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000


,splash,chance,funny,square,help,lonely,queer,gently,hill,mercia,...,frightened,float,dormouse,sacred,thoroughly,apple,hospital,only,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [85]:
word_counts2.text_source.value_counts()

Whitman    5709
Carroll    1669
Name: text_source, dtype: int64

Let's retry the logistic regression again with the new data / features

In [87]:
# Add a string sentence
word_counts2['str_sentence'] = [x.text for x in word_counts2['text_sentence']]

# Add the other features we have from before
word_counts2['sent_len'] = [len(i) for i in word_counts2['str_sentence']]

# How many of each type of word is there?
word_counts2['cnt_verbs'] = [sum([1 for x in j if x.pos_ == 'VERB']) for j in word_counts2.text_sentence]
word_counts2['cnt_adj'] = [sum([1 for x in j if x.pos_ == 'ADJ']) for j in word_counts2.text_sentence]
word_counts2['cnt_prop'] = [sum([1 for x in j if x.pos_ == 'PROPN']) for j in word_counts2.text_sentence]
word_counts2['cnt_punct'] = [sum([1 for x in j if x.pos_ == 'PUNCT']) for j in word_counts2.text_sentence]
word_counts2['cnt_adv'] = [sum([1 for x in j if x.pos_ == 'ADV']) for j in word_counts2.text_sentence]
word_counts2['cnt_nouns'] = [sum([1 for x in j if x.pos_ == 'NOUN']) for j in word_counts2.text_sentence]
word_counts2['crude_sentiment'] = [j.sentiment for j in word_counts2.text_sentence]

In [88]:
Y = word_counts2['text_source']
X = np.array(word_counts2.drop(['text_sentence','text_source', 'str_sentence'], 1))

In [89]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y,test_size=0.4, random_state=0)

Running this improves the score a little bit, looking at it though, most of the issues are coming from the perspective of the Carroll features. I wonder if we balance out the classes, it will fix it.

In [111]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train2, y_train2)

y_pred2 = lr.predict(X_test2)

print('Training set score:', lr.score(X_train2, y_train2))
print('\nTest set score:', lr.score(X_test2, y_test2))

from sklearn.model_selection import cross_val_score
print('\nCross Val score:')
print(cross_val_score(lr, X_train2, y_train2, cv = 5))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test2, y_pred2))

Training set score: 0.9640759150474469

Test set score: 0.9308943089430894

Cross Val score:
[0.92776524 0.91751412 0.93107345 0.92429379 0.92881356]

Classification Report:
               precision    recall  f1-score   support

     Carroll       0.93      0.76      0.83       674
     Whitman       0.93      0.98      0.96      2278

   micro avg       0.93      0.93      0.93      2952
   macro avg       0.93      0.87      0.89      2952
weighted avg       0.93      0.93      0.93      2952



In [92]:
word_counts2.columns

Index(['splash', 'chance', 'funny', 'square', 'help', 'lonely', 'queer',
       'gently', 'hill', 'mercia',
       ...
       'text_source', 'str_sentence', 'sent_len', 'cnt_verbs', 'cnt_adj',
       'cnt_prop', 'cnt_punct', 'cnt_adv', 'cnt_nouns', 'crude_sentiment'],
      dtype='object', length=1599)

In [93]:
word_s = word_counts2.sample(frac=1, random_state=1)

carroll_s = word_s.loc[word_s['text_source']=='Carroll'][:1669]
whitman_s = word_s.loc[word_s['text_source']=='Whitman'][:1669]

words = pd.concat([carroll_s, whitman_s])
words = words.sample(frac=1, random_state=40)

In [94]:
Y = words['text_source']
X = np.array(words.drop(['text_sentence','text_source', 'str_sentence'], 1))

In [95]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, Y,test_size=0.4, random_state=0)

In [102]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train3, y_train3)

y_pred3 = lr.predict(X_test3)

print('Training set score:', lr.score(X_train3, y_train3))
print('\nTest set score:', lr.score(X_test3, y_test3))

from sklearn.model_selection import cross_val_score
print('\nCross Val score:')
print(cross_val_score(lr, X_train3, y_train3, cv = 5))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test3, y_pred3))

Training set score: 0.9655344655344655

Test set score: 0.9041916167664671

Cross Val score:
[0.9201995  0.89775561 0.8553616  0.915      0.89223058]

Classification Report:
               precision    recall  f1-score   support

     Carroll       0.91      0.89      0.90       661
     Whitman       0.90      0.92      0.91       675

   micro avg       0.90      0.90      0.90      1336
   macro avg       0.90      0.90      0.90      1336
weighted avg       0.90      0.90      0.90      1336



So that didn't improve the overall score of the data but did make prediciting Carroll much more effective in comparison to predicting Whitman. Also looks as if the model with the full dataset is less prone to overfitting than with the balanced dataset.

### Tune the Model

In [103]:
# Grid Search CV for decision tree
from sklearn.model_selection import GridSearchCV

#GridSearchCV for random forest 
param_grid = {'C':[1e9,.5,1,3,5,10], 'max_iter':[50,100,150,300,500], 'penalty':['l1','l2']}

# Start the grid search again
grid_DT = GridSearchCV(lr, param_grid, cv=3, verbose=1, n_jobs=-1)

grid_DT.fit(X_train3, y_train3)

# summarize the results of the grid search
# View the accuracy score
print('Best score for data:')
print(grid_DT.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.2s


Best score for data:
{'C': 3, 'max_iter': 50, 'penalty': 'l2'}


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   41.8s finished


In [104]:
# Grid Search CV for decision tree
from sklearn.model_selection import GridSearchCV

#GridSearchCV for random forest 
param_grid = {'C':[3], 'max_iter':[10,15,25,30,45,50], 'penalty':['l2']}

# Start the grid search again
grid_DT = GridSearchCV(lr, param_grid, cv=3, verbose=1, n_jobs=-1)

grid_DT.fit(X_train3, y_train3)

# summarize the results of the grid search
# View the accuracy score
print('Best score for data:')
print(grid_DT.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best score for data:
{'C': 3, 'max_iter': 25, 'penalty': 'l2'}


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    3.4s finished


In [106]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=3, max_iter=25, penalty='l2')
train = lr.fit(X_train3, y_train3)

y_pred3 = lr.predict(X_test3)

print('Training set score:', lr.score(X_train3, y_train3))
print('\nTest set score:', lr.score(X_test3, y_test3))

from sklearn.model_selection import cross_val_score
print('\nCross Val score:')
print(cross_val_score(lr, X_train3, y_train3, cv = 5))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test3, y_pred3))

Training set score: 0.985014985014985

Test set score: 0.9094311377245509

Cross Val score:


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[0.91521197 0.90024938 0.85785536 0.9175     0.89473684]

Classification Report:
               precision    recall  f1-score   support

     Carroll       0.92      0.90      0.91       661
     Whitman       0.90      0.92      0.91       675

   micro avg       0.91      0.91      0.91      1336
   macro avg       0.91      0.91      0.91      1336
weighted avg       0.91      0.91      0.91      1336



### Test the new paramaters on the full dataset
The new parameters don't change all that much on the outcome of the data here.

In [110]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=3, max_iter=25, penalty='l2')
train = lr.fit(X_train2, y_train2)

y_pred3 = lr.predict(X_test2)

print('Training set score:', lr.score(X_train2, y_train2))
print('\nTest set score:', lr.score(X_test2, y_test2))

from sklearn.model_selection import cross_val_score
print('\nCross Val score:')
print(cross_val_score(lr, X_train2, y_train2, cv = 5))

from sklearn.metrics import classification_report
print('\nClassification Report:\n',classification_report(y_test2, y_pred2))

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training set score: 0.9681427925892454

Test set score: 0.9380081300813008

Cross Val score:


/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[0.93679458 0.93107345 0.93672316 0.92655367 0.93220339]

Classification Report:
               precision    recall  f1-score   support

     Carroll       0.93      0.76      0.83       674
     Whitman       0.93      0.98      0.96      2278

   micro avg       0.93      0.93      0.93      2952
   macro avg       0.93      0.87      0.89      2952
weighted avg       0.93      0.93      0.93      2952



/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
